# 结果评估(Evaluation)
由于自然语言的不可预测性和可变性，评估LLM的输出是否正确有些困难，langchain 提供了一种方式帮助我们去解决这一难题。

- Evaluation是对应用程序的输出进行质量检查的过程
- 正常的、确定性的代码有我们可以运行的测试，但由于自然语言的不可预测性和可变性，判断 LLM 的输出更加困难
- langchain 提供了一种方式帮助我们去解决这一难题
- 对于QApipline 生成的summary进行质量审查
- 对 Summary pipline的结果进行检查

In [1]:


# here put the import lib
from typing import Any, List, Mapping, Optional, Dict
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from zhipuai import ZhipuAI

import os

# 继承自 langchain.llms.base.LLM
class ZhipuAILLM(LLM):
    # 默认选用 glm-3-turbo
    model: str = "glm-3-turbo"
    # 温度系数
    temperature: float = 0.1
    # API_Key
    api_key: str = "acf4f9247da5e232fbe056b14b35fd9b.uWW0WvWqwWUYjhzQ"
    
    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
        client = ZhipuAI(
            api_key = self.api_key
        )

        def gen_glm_params(prompt):
            '''
            构造 GLM 模型请求参数 messages

            请求参数：
                prompt: 对应的用户提示词
            '''
            messages = [{"role": "user", "content": prompt}]
            return messages
        
        messages = gen_glm_params(prompt)
        response = client.chat.completions.create(
            model = self.model,
            messages = messages,
            temperature = self.temperature
        )

        if len(response.choices) > 0:
            return response.choices[0].message.content
        return "generate answer error"


    # 首先定义一个返回默认参数的方法
    @property
    def _default_params(self) -> Dict[str, Any]:
        """获取调用API的默认参数。"""
        normal_params = {
            "temperature": self.temperature,
            }
        # print(type(self.model_kwargs))
        return {**normal_params}

    @property
    def _llm_type(self) -> str:
        return "Zhipu"

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {**{"model": self.model}, **self._default_params}

In [2]:
llm = ZhipuAILLM()

In [4]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Eval
from langchain.evaluation.qa import QAEvalChain

In [5]:
# 加载文档
loader = TextLoader("./data/wonderland.txt", 'utf-8')
doc = loader.load()

print (f"You have {len(doc)} document")
print (f"You have {len(doc[0].page_content)} characters in that document")

You have 1 document
You have 13637 characters in that document


In [6]:
# 文档切分
text_spliter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=400)
docs = text_spliter.split_documents(doc)

# Get the total number of characters so we can see the average later
num_total_characters = sum([len(x.page_content) for x in docs])

print (f"Now you have {len(docs)} documents that have an average of {num_total_characters / len(docs):,.0f} characters (smaller pieces)")

Now you have 6 documents that have an average of 2,272 characters (smaller pieces)


In [8]:
# 向量化处理
embeddings = SentenceTransformerEmbeddings(model_name="D:/code/models/M3E/xrunda/m3e-base")

# 向量化数据库
vectorstores = FAISS.from_documents(doc, embeddings)

In [10]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=vectorstores.as_retriever(),
    input_key="question"
)
# 注意这里的 input_key 参数，这个参数告诉了 chain 我的问题在字典中的哪个 key 里
# 这样 chain 就会自动去找到问题并将其传递给 LLM

In [11]:
question_answers = [
    {'question' : "Which animal give alice a instruction?", 'answer' : 'rabbit'},
    {'question' : "What is the author of the book", 'answer' : 'Elon Mask'}
]

In [12]:
pred = chain.apply(question_answers)
pred
# 使用LLM模型进行预测，并将答案与我提供的答案进行比较，这里信任我自己提供的人工答案是正确的

D:\Python\envs\py310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.apply` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use batch instead.
  warn_deprecated(


[{'question': 'Which animal give alice a instruction?',
  'answer': 'rabbit',
  'result': 'The White Rabbit gave Alice instructions.'},
 {'question': 'What is the author of the book',
  'answer': 'Elon Mask',
  'result': 'The author of the book you\'ve described is Lewis Carroll. He wrote "Alice\'s Adventures in Wonderland," which is the story of Alice and her strange and fantastical experiences in Wonderland. If you have any more questions about the book or its characters, feel free to ask!'}]

In [13]:
# Start your eval chain
eval_chain = QAEvalChain.from_llm(llm)

graded_outputs = eval_chain.evaluate(question_answers,
                                     pred,
                                     question_key="question",
                                     prediction_key="result",
                                     answer_key='answer')

In [14]:
graded_outputs

[{'results': 'GRADE: CORRECT'}, {'results': 'GRADE: INCORRECT'}]